# SageMaker Batch Transform: Offline Inference
In the case that you don't want to use an endpoint driven option and just want offline inference on a dataset you can use [SageMaker Batch Transform](https://docs.aws.amazon.com/sagemaker/latest/dg/batch-transform.html). In this example we'll take a sample [Distillbert model](https://huggingface.co/distilbert/distilbert-base-uncased-finetuned-sst-2-english) (Apache 2.0 license) and showcase how you can create a Transform Job.

### Additional Resources/Credits
- [HuggingFace Workshop](https://github.com/philschmid/huggingface-sagemaker-workshop-series/blob/main/workshop_2_going_production/lab2_batch_transform.ipynb)
- [HF Custom Inference Script Implementation](https://github.com/huggingface/notebooks/blob/main/sagemaker/17_custom_inference_script/code/inference.py)
- [Batch Transform Examples](https://github.com/RamVegiraju/SageMaker-Deployment/tree/master/BatchTransform)

## Setup

In [ ]:
!pip install --upgrade --quiet sagemaker jsonlines

In [ ]:
from sagemaker.huggingface import HuggingFaceModel
import sagemaker

# setup role and sm session and default bucket
role = sagemaker.get_execution_role()
sagemaker_session = sagemaker.Session()
default_bucket = sagemaker_session.default_bucket()


# HuggingFace Model Object
hub = {
	'HF_MODEL_ID':'distilbert/distilbert-base-uncased-finetuned-sst-2-english',
	'HF_TASK':'text-classification'
}

huggingface_model = HuggingFaceModel(
    transformers_version='4.37.0',
    pytorch_version='2.1.0',
    py_version='py310',
    env=hub,
    role=role
)

## Create Test Dataset
We make a mock dataset with 50 copies of the same input payload, adjust this accordingly for your use-case. In this case we have a JSON file for each datapoint, optionally you can also have a singular JSONLines with all 50, ensure to adjust the transformers mime types in that scenario like in the HF workshop above.

In [ ]:
import boto3
import json

# s3 bucket subfolder with test data
prefix = 'batch-input/'

# JSON payload
payload = {"inputs": "I am super happy right now."}

# S3 client
s3 = boto3.client('s3')

# Upload json objects for each sample data point
for i in range(50):
    file_content = json.dumps(payload)
    file_name = f"{prefix}input_{i:03}.json"
    
    s3.put_object(
        Bucket=default_bucket,
        Key=file_name,
        Body=file_content,
        ContentType='application/json'
    )
    print(f"Uploaded: s3://{default_bucket}/{file_name}")
    
input_data_path = f"s3://{default_bucket}/{prefix}"

In [ ]:
!aws s3 ls {input_data_path} #verify input data points there

## Transform Job
Here we kick off a transform job, adjust instance type, count, and also the mime type depending on the data format you're working with.

In [ ]:
# where to dump output results
out_prefix = "transform-results/"
output_data_path = f"s3://{default_bucket}/{out_prefix}"

# transformer object
transformer = huggingface_model.transformer(
    instance_count=1,
    instance_type='ml.m5.large',
    strategy='SingleRecord',
    assemble_with='Line',
    output_path=output_data_path,
    accept="application/json"
)

# Feed the test data
transformer.transform(input_data_path, 
                      content_type="application/json", 
                      split_type='Line')

# Wait for job to complete
print("Waiting for transform job: " + transformer.latest_transform_job.job_name)
transformer.wait()
output = transformer.output_path

## Parse Output Results
Pull down the results from S3 and read.

In [ ]:
# output dataset s3 path
output_results = transformer.output_path
output_results

In [ ]:
!aws s3 ls {output_results}

In [ ]:
# copy over the s3 output data results locally
import subprocess
subprocess.run(f"mkdir -p results && aws s3 cp {output_results} ./results/ --recursive", shell=True)

In [ ]:
# read a sample output data point
import json
with open('results/input_005.json.out', 'r') as f:
    data = json.load(f)
print(data)